# Information Coding 2023-24

## Lab assignment 1 (Lab session 2, 28-09-2023)
## Symmetric Cryptography

### (Estimated working time: 2 hours lab + 2 hours home);
	
### Student names:
#### Javier Cirugeda Bugallo

## INSTRUCTIONS
- Practices are to be done in groups of two (or one). 
- This solved practice has to be uploaded in Moodle before October 9th, 23:55h. 
- The submission must be done by the first author (alphabetical order).
- **Name the file with authors' names.**

This practice accounts for 6% of final grade for both autors.

### Grading items:
- Defined functions 50% (eficiency, clarity, code documentation)
- Exercise solving 40%
- Mathematical correction, clarity of results, language precission, 10%.
- **To get any credit, physical presence in the practice session is a must.**

Unless otherwise stated, we shall use the following alphabet (Castilian with blank space)

In [2]:
ALFA = " ABCDEFGHIJKLMNÑOPQRSTUVWXYZ"
len(ALFA)

28

##  1 (1 pt) Complete the introduction on Affine Matricial Cryptosystem

in the affine matricial cryptosystem, the key is a pair $K=[M, b]$, where $M$ is a square $n \times n$ matrix and $b$ is a row vector (a $1 \times n$ matrix), both with coefficients In $Z_q$ ($q$ is the size of the alphabet)
	
The plaintext must be divided in blocks of size $n$.

With each block, a row vector $x$ of size $n$ is built, using the numerical equivalent of each symbol in the block.

The vector is encryted using the function $c_k(x):=mod((x \cdot M+b),q)$
	
The ciphertext is the concatenation of all the encrypted vectors, after translating them back to symbols.

The conditions that $M$ and $b$ must verify so that $c_k$ is a valid encryption function are just That $M$ is invertible modulo $q$, that is, $gcd(det(M),q)=1$

the decryption function is $d_k(y)=mod(y.M^{-1}-b\cdot M^{-1},q)$
	
Affine matrix cryptosystem is interesting for us because it is a general formulation of what we have seen:
- If the independent term is the zero vector we have the matricial cryptosystem
- If the order $n$ of $M$ and $b$ is $n=1$, we have the affine cryptosystem. If $M=[1]$, we have Caesar's.

Find information on the Vigenére cipher. Reason that it is just an affine matricial cryptosystem. Who are $M$ and $b$ in this case?


##  2 (2 pt) Message to Vectors and Vectors to Message

To use an affine matrix cryptosystem of order $n$, we must begin by converting the message into a list of vectors of size n (whose entries are the numerical equivalents of the corresponding symbols). After encrypting, the resulting vectors must be transformed back into a string.



##   2.1 StringToVectors

Write a function StringToVectors

Input:
- An integer, n (size of blocks)
- A string, A (the alphabet)
- A string, m (the message)

Output:
- A list of row vectors of size n, containing the numerical equivalents of the symbols in the message.
	
*Hint: Remember that the numerical equivalent of a symbol is its position in the alphabet, minus 1.*

**Important:**
- The procedure must control the correction of entry data (no repeated symbols in the alphabet, no symbols in the message except for the ones in the alphabet...).
- Afterwards, the numerical equivalent to each symbol in the message must be obtained. The result is a list of integers. If the size of this list is not a multiple of n, fill with zeros.
- Finally, we put those integers in vectors, in groups of n, and put the vectors into a list, which shall be the output.

In [3]:
# To create vectors

import sympy as sp

v=sp.Matrix(1,3,[2,5,3])
print(v)

Matrix([[2, 5, 3]])


In [4]:
'''def StringToVectors(n, A, m):
    output = []
    for char in m:
        output.append(A.find(char))
    return output
StringToVectors(1, ALFA, "Hola")'''
import sympy

def StringToVectors(n, A, m):
    # Step 1: Check input data validity
    if len(set(A)) != len(A):
        raise ValueError("Alphabet contains repeated symbols")
    if any(symbol not in A for symbol in m):
        raise ValueError("Message contains symbols not in the alphabet")

    # Step 2: Calculate the numerical equivalent of each symbol
    numerical_equivalents = [A.index(symbol) for symbol in m]

    # Step 3: Fill with zeros to make the list a multiple of n
    remainder = len(numerical_equivalents) % n
    if remainder != 0:
        numerical_equivalents.extend([0] * (n - remainder))

    # Step 4: Organize into row vectors of size n
    vectors = [numerical_equivalents[i:i + n] for i in range(0, len(numerical_equivalents), n)]

    return vectors


## 2.2 VectorsToString

Write a function VectorsToString, that performs the inverse procedure to StringToVectors 

Input:
- An integer, n (size of blocks)
- A string, A (the alphabet)
- A list, lvec, of row vectors of size n, containing integers (in the range 0..len(A)-1) 

Output:
- A string, m (the symbols corresponding to the numbers in the list)

In [5]:
def VectorsToString(n, A, lvec):
    # Step 1: Check the validity of the input data
    if len(set(A)) != len(A):
        raise ValueError("Alphabet contains repeated symbols")

    # Step 2: Convert row vectors back to a list of integers
    numerical_equivalents = [elem for vector in lvec for elem in vector]

    # Step 3: Convert numerical equivalents to symbols
    message = ''.join([A[num] for num in numerical_equivalents])

    return message

## 2.3 Testing

Test the procedures above with different situations (wrong entry data, different alphabets, using lists instead of vector (matrices),...).

In [6]:
n = 3
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
message = "HELLO"
result = StringToVectors(n, alphabet, message)
print(result)
msg = VectorsToString(n, alphabet, result)
print(msg)

[[7, 4, 11], [11, 14, 0]]
HELLOA


## 3 (3 pt) Affine Matrix Cryptosystem

##   3.1 AMC

Write a function AMC implementing the affine matrix cryptosystem:
Input:
- A string m (the plaintext)
- A string, A (the alphabet)
- A square matrix M
- A row vector b 

Output:
- A string, c (the cryptogram obtained from m using the key [M,b])

*Notes:*
- The procedure must check the sizes of matrix and vector and the validity of the key.
- The same procedure can be used to decrypt, by using the appropiate inverse key.

In [19]:
# To multiply matrices and vectors
#M=sp.Matrix(2,2,[1,2,3,4]) # or M=sp.Matrix([[1,2],[3,4]])
#v=sp.Matrix(1,2,[3,1])
#print(v*M)

def is_square_matrix(matrix):
    return matrix.shape[0] == matrix.shape[1]

def validate_key(alphabet, M, b):
    if not is_square_matrix(M):
        raise ValueError("Matrix M must be square.")
    if not M.shape[0] == len(b):
        raise ValueError("Invalid vector lenght.")
    if sp.det(M) % len(alphabet) == 0:
        raise ValueError("Invalid matrix.")


def AMC_encrypt(m, A, M, b):
    validate_key(A, M, b)
    numerical_equivalents = [A.index(char) for char in m]
    encrypted_numericals = (b * M) % len(A)
    return encrypted_numericals
    
    
M = sp.Matrix(3, 3, [3, 2, 3, 4, 5, 6, 7, 8, 8])


plaintext = "HELLO"
ciphertext = ""
n = 3
result = []
print("> Original text: " + plaintext)
# CYPHER
for vector in StringToVectors(n, ALFA, plaintext):
    result.append(list(AMC_encrypt(plaintext, ALFA, M, sp.Matrix(1, n, vector))))
'''
print("--------------------------------------------------------------------")
print(AMC_encrypt(plaintext, ALFA, M, sp.Matrix(1, n, vector)))
print(list(AMC_encrypt(plaintext, ALFA, M, sp.Matrix(1, n, vector))))
print(result)
print("--------------------------------------------------------------------")
'''
ciphertext = VectorsToString(n, ALFA, result)
print("Ciphertext:", ciphertext)

# DECYPHER
invM = M.inv_mod(len(ALFA))
orgText = []
for vector in StringToVectors(n, ALFA, ciphertext):
    orgText.append(list(AMC_encrypt(ciphertext, ALFA, invM, sp.Matrix(1, n, vector))))
recoveredtext = VectorsToString(n, ALFA, orgText)
print("> Recovered text: " + recoveredtext)

> Original text: HELLO
Ciphertext: OXJOSS
> Recovered text: HELLO 


In [48]:
M=sp.Matrix(1,4,[1,2,3,4])
print(list(M))

[1, 2, 3, 4]


In [ ]:
##  3.2 Examples

Use AMC to encrypt and decrypt, when possible,
the following messages, with the given alphabet and keys:

In [14]:
m="patata"
Alf="patata"
M=sp.Matrix([[4,0],[0,1]])
b=sp.Matrix([[0,2]])

In [16]:
m="PROBANDO  AMC"
# alphabet ALFA 
M1=sp.Matrix([[5,0],[0,1]])
b1=sp.Matrix([[0, 2]])
              
M2=sp.Matrix([[3,0],[0,1]])
b2=sp.Matrix([[0, 2, 2]])

M3=sp.Matrix([[3,0],[1,1]])
b3=sp.Matrix([[0,2]])

In [17]:
m1="CIPHER THIS TEXT WITH CAESAR"  #  alphabet ALFA, Caesar cipher, key:3

m2="CIPHER THIS TEXT WITH AFFINE" # alphabet ALFA, affine cipher, key [3,2]

m3="CIPHER THIS TEXT WITH VIGENERE" # alphabet ALFA, Vigenére cipher, key "CLAVE"

In [59]:
# To compute the modular inverse of a matrix\n",

def matModInv(M,n):
    if sp.det(M) == 0:
        raise ValueError("The matrix doesn't have an inverse")
    output=sp.transpose(M.cofactor_matrix())*pow(sp.det(M),-1,n)
    return(output%n)

# Note that pow(x,-1,n) to compute modular inverses works only from python 3.8+
    
# sp.transpose(M.cofactor_matrix()) can also be computed by M.adjugate()\n",

M=sp.Matrix([[1,2],[3,4]])
N=matModInv(M,11)

print(N)
print(M*N%11)

Matrix([[9, 1], [7, 5]])
Matrix([[1, 0], [0, 1]])


## 4 (4 pt) Cryptanalysis

## 4.1 Known plaintext attack

Exercise 1.13 from the notes:

A linear matrix cryptosystem of size $2\times 2$ has been used. Alphabet is ALFA. We know that the last five letters of the message were "CARLA". Get the plaintext.

*Hint: You can use the modular inverse to calculate the modular inverse of $x$ $mod q$, and solve the systems of equations $mod 28$ that shall appear.*


In [24]:
ciphertext = "WSDYLYNNDWKKSLRSDWEEMNXXTJCK"

M = sp.Matrix(2, 2, [23, 23, 5, 4]) # Calculations done by hand using a matrix ecuation
n = 2
result = []
for vector in StringToVectors(n, ALFA, ciphertext):
    result.append(list(AMC_encrypt(ciphertext, ALFA, M, sp.Matrix(1, n, vector))))
#print(result)
plaintext = VectorsToString(n, ALFA, result)
print("Plaintext:", plaintext)

Plaintext: HOY NO NOS PODEMOS VER CARLA


##   4.2 Relative frequency

Break the cryptogram cr.

The alphabet ALFA and an affine cipher have been used.

You shall need to count the relative frequency of each symbol...

In [41]:
cr = "TUPRUYMEUPYUVÑFCUPNCFWYUPÑEUIUPIÑ UCJNPTUPENWÑCIUPWÑN  UPYÑCIFUTPRÑNPIFENHUIUPQM PU JAÑ PEDAN ZFÑE"

import collections

# Count the frequency of each symbol in the ciphertext
symbol_counts = collections.Counter(cr)

# Calculate the total number of symbols in the ciphertext
total_symbols = len(cr)

# Calculate the relative frequency of each symbol
relative_frequencies = {symbol: count / total_symbols for symbol, count in symbol_counts.items()}

# Sort the symbols by frequency (highest to lowest)
sorted_symbols = sorted(relative_frequencies.items(), key=lambda x: x[1], reverse=True)

# Print the symbols ordered by frequency
for symbol, frequency in sorted_symbols:
    print(f"Symbol: {symbol}, Relative Frequency: {frequency:.4f}")
    
'''
result = []
for vector in StringToVectors(len(cr), ALFA, cr):
    result.append(vector)
    
result2 = []
result3 = []
for symbol in result[0]:
    result2.append((5 * symbol - 5 * 21) % len(ALFA))
result3.append(result2)
plaintext = VectorsToString(len(cr), ALFA, result3)
print("Plaintext:", plaintext)
'''

Symbol: U, Relative Frequency: 0.1633
Symbol: P, Relative Frequency: 0.1429
Symbol: Ñ, Relative Frequency: 0.0918
Symbol: N, Relative Frequency: 0.0714
Symbol:  , Relative Frequency: 0.0714
Symbol: E, Relative Frequency: 0.0612
Symbol: I, Relative Frequency: 0.0612
Symbol: F, Relative Frequency: 0.0510
Symbol: C, Relative Frequency: 0.0510
Symbol: Y, Relative Frequency: 0.0408
Symbol: T, Relative Frequency: 0.0306
Symbol: W, Relative Frequency: 0.0306
Symbol: R, Relative Frequency: 0.0204
Symbol: M, Relative Frequency: 0.0204
Symbol: J, Relative Frequency: 0.0204
Symbol: A, Relative Frequency: 0.0204
Symbol: V, Relative Frequency: 0.0102
Symbol: H, Relative Frequency: 0.0102
Symbol: Q, Relative Frequency: 0.0102
Symbol: D, Relative Frequency: 0.0102
Symbol: Z, Relative Frequency: 0.0102


'\nresult = []\nfor vector in StringToVectors(len(cr), ALFA, cr):\n    result.append(vector)\n    \nresult2 = []\nresult3 = []\nfor symbol in result[0]:\n    result2.append((5 * symbol - 5 * 21) % len(ALFA))\nresult3.append(result2)\nplaintext = VectorsToString(len(cr), ALFA, result3)\nprint("Plaintext:", plaintext)\n'

## 4.3 Bonus track

- Solve exercise 1.9 d) from the notes.

- What extra functions would you need to implement in order to solve exercise 1.11? Can you solve it?
